In [1]:
import os
import cv2
import shutil
import deeplabcut
import numpy as np
import pandas as pd
import subprocess
from datetime import date
from joblib import Parallel, delayed

2023-03-31 20:54:43.445461: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 20:54:43.554350: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-31 20:54:43.880398: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yramakrishna/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2023-03-31 20:54:43.880443: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic l

Loading DLC 2.3.1...


In [2]:
if not os.path.exists('All_vids'):
        os.makedirs('All_vids')
os.chdir('All_vids')

In [3]:
#enter path of folder (including folder name) containing all your videos of interest
vid_dir = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/Videos'
to_dir = os.path.join(os.getcwd(), os.path.basename(vid_dir))
shutil.move(vid_dir, to_dir)

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Videos'

In [4]:
os.path.dirname(to_dir)

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids'

In [5]:
#Soring converted videos in a new folder
if not os.path.exists('Conv_vids'):
        os.makedirs('Conv_vids')

In [6]:
os.getcwd()

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids'

In [7]:
input_file = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Videos/ken_20210503_0911_XBI19_reversal_learning_foraging_SC3_DCS_frontcamera.mov'
output_file = os.path.splitext(os.path.basename(input_file))[0].lstrip('.') 
output_file
os.path.join(os.path.dirname(os.path.dirname(input_file)), '/Conv_vids', output_file +'_conv.mp4')

'/Conv_vids/ken_20210503_0911_XBI19_reversal_learning_foraging_SC3_DCS_frontcamera_conv.mp4'

In [10]:
#Converting .mov videos to .mp4
def conv_mp4(input_file):
    out_name = os.path.splitext(input_file)[0].lstrip('.') 
    print(out_name)
    output_file = out_name + '.mp4'
    # print(input_file)
    # print(output_file)
    # Run FFmpeg command to convert MOV to MP4
    command = f"ffmpeg -i {input_file} -c:v libx264 -preset medium -crf 23 -c:a aac -b:a 128k {output_file}"
    subprocess.call(command, shell=True)
    new = os.path.join(os.path.dirname(os.path.dirname(output_file)), 'Conv_vids', os.path.basename(output_file))
    shutil.move(output_file, new)
    # print('Convertion complete - ', os.path.basename(output_file))

In [9]:
for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'Videos')):
    if file.endswith(".mov"):
        # print(file)
        path=os.path.join(os.path.join(os.path.dirname(to_dir), 'Videos', file))
        conv_mp4(path)
        # print(path)

/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Videos/vin_20210604_0907_XBI19_reversal_learning_foraging_SC3_DCS_front_camera
/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Videos/nil_20210614_XBI19


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

KeyboardInterrupt: 

In [ ]:
# if not os.path.exists('NDSConv_vids'):
#         os.makedirs('NDSConv_vids')

In [ ]:
if not os.path.exists('GC_Conv_vids'):
        os.makedirs('GC_Conv_vids')
os.chdir('GC_Conv_vids')

In [ ]:
os.getcwd()

In [ ]:
#os.listdir(os.path.join(os.path.dirname(to_dir), 'DSConv_vids'))
# os.path.join(os.path.dirname(to_dir), 'NDSConv_vids', os.path.basename(dsv)) 

In [ ]:
#cropping video and Gamma correction
def downsize_gc(convid):
    dsv = deeplabcut.DownSampleVideo(convid, width=320, height=240)
    # Load the video file
    video = cv2.VideoCapture(dsv)

    # Get the frame rate and total number of frames
    fps = int(video.get(cv2.CAP_PROP_FPS))
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    #*********************************************

    # Set the start and end points to crop out
    start_sec = 0  # Crop out the first x seconds
    end_sec = 0 # Crop out the last y seconds

    #Set gamma correction value
    gamma = 2

    #**********************************************

    start_frame = int(start_sec * fps)
    end_frame = int((total_frames / fps) - end_sec) * fps

    # Set the video writer
    output_name = os.path.splitext(os.path.basename(convid))[0].lstrip('.') + '_GC.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_name, fourcc, fps, (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    # Set the current frame number to the start frame
    video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Loop through the frames and write them to the output video
    for i in range(start_frame, end_frame):
        ret, frame = video.read()
        if not ret:
            break

        # Normalize the pixel values to the range [0, 1]
        frame_normalized = frame.astype(np.float32) / 255.0

        # Apply gamma correction
        frame_corrected = np.power(frame_normalized, 1/gamma)

        # Scale the pixel values back to the range [0, 255]
        frame_scaled = (frame_corrected * 255.0).astype(np.uint8)

        out.write(frame_scaled)
    new = os.path.join(os.path.dirname(os.path.dirname(dsv)), 'GC_Conv_vids', os.path.basename(dsv))
    shutil.move(dsv, new)

    # Release the video objects
    video.release()
    out.release()
    cv2.destroyAllWindows()

In [ ]:
# vid_names = []
for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'Conv_vids')):
    if file.endswith(".mp4"):
        path=os.path.join(os.path.join(os.path.dirname(to_dir), 'Conv_vids', file))
        downsize_gc(path)

In [ ]:
# os.path.join(os.path.dirname(os.path.dirname(input_file)), 'All_vids/Conv_vids', out_name +'_conv.mp4')
# out_name = os.path.splitext(os.path.basename(path))[0].lstrip('.') + '.mp4'
# out_name

In [ ]:
os.chdir('..')
os.getcwd()

In [ ]:
folder = [] 
for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'GC_Conv_vids')):
    if file.endswith(".mp4"):
        path=os.path.join(os.path.join(os.path.dirname(to_dir), 'GC_Conv_vids', file))
        folder.append(path)
print(folder)
        
# folder2 = [] 
# for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'Conv_vids')):
#     if file.endswith(".mp4"):
#         path=os.path.join(os.path.join(os.path.dirname(to_dir), 'Conv_vids', file))
#         folder2.append(path)
        
# print(folder, folder2)

In [ ]:
project_name = 'DLC'
your_name = 'VK'

In [ ]:
process(folder, project_name, your_name)

# all_fold = [folder2, folder]
# print(all_fold)

In [ ]:
def process(fold, project_name, your_name):
    file = fold[0]
    print(file)
    bodyparts = ['RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'RightEye_Pupil', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'LeftEye_Pupil',
                 'OutlineTop_Mid','RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']
    videotype = os.path.splitext(file)[-1].lstrip('.')  # or MOV, or avi, whatever you uploaded!
    video_down = folder
    name_fold = 'Analyse_' + os.path.basename(os.path.dirname(file))
    
    model_options = deeplabcut.create_project.modelzoo.Modeloptions
    model_selection = 'primate_face'

    config_path, train_config_path = deeplabcut.create_pretrained_project(
        project_name,
        your_name,
        video_down,
        videotype=videotype,
        model=model_selection,
        analyzevideo=True,
        createlabeledvideo=False,
        copy_videos=False,
    )

    edits = {
        'dotsize': 1.5,  # size of the dots!
        'pcutoff': 0.4,  # the higher, the more conservative the plotting!
    }
    deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)
    project_path = os.path.dirname(config_path)
    full_video_path = []
    for i in fold:
        full_video_path.append(os.path.join(project_path,'videos', os.path.basename(i)))

    # filter predictions (should already be done above ;):
    deeplabcut.filterpredictions(config_path, full_video_path, videotype=videotype)

    # re-create the video with your edits!
    # deeplabcut.CropVideo(config_path, full_video_path, 'crop', 
    deeplabcut.create_labeled_video(config_path, full_video_path, videotype=videotype, displayedbodyparts=bodyparts, draw_skeleton = True, filtered=True)
    
    

In [ ]:
# def main():
#     Parallel(n_jobs=1)(delayed(process)(fold, project_name, your_name) for fold in all_fold)


In [ ]:
# main()

In [ ]:

# for fold in all_fold:
#     name_fold = 'Analyse_' + os.path.basename(os.path.dirname(fold[0]))
#     print(name_fold)
#     print(fold)
#     if not os.path.exists(name_fold):
#         os.makedirs(name_fold)
#     os.chdir(name_fold)
#     print(os.getcwd())
#     process(fold, project_name, your_name)
#     os.chdir('..')
#     print(os.getcwd())
    

In [ ]:

# proj_fold

In [ ]:
h5files = []
vid_to_crop = []
today = str(date.today())
proj_fold = project_name+'-'+your_name+'-'+today

target = os.path.join(os.getcwd(),proj_fold,'videos')
print(target)
for file in os.listdir(target):
    if file.endswith("filtered.h5"):
        h5files.append(os.path.join(target,file))
for file in os.listdir(target):   
    if file.endswith("labeled.mp4"):
        vid_to_crop.append(os.path.join(target,file))
        
# print(h5files, vid_to_crop)
# videotype = os.path.splitext(file)[-1].lstrip('.')

In [ ]:
if len(h5files)==len(vid_to_crop):
    for i in h5files:
        found = ''
        for j in vid_to_crop:
            x = os.path.splitext(i)[0].rstrip('_filtered.h5')
            y = os.path.splitext(j)[0].rstrip('filtered_labeled.mp4')
            if x==y:
                found = j
        k = os.path.basename(i)
        l = 'DLC_re' + k.split('re')[1].rstrip('_filtered')
        # print(l)
        crop_it(i,j, l)

In [ ]:
# DLC_resnet50_DLCMar31shuffle1_1030000
# fl_pth = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_video/DLC_GazeXBI-anc-2023-03-31/videos/CropGrDLC_resnet50_DLC_GazeXBIMar31shuffle1_1030000_filtered.h5'

# vid_pth = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_video/DLC_GazeXBI-anc-2023-03-31/videos/CropGrDLC_resnet50_DLC_GazeXBIMar31shuffle1_1030000_filtered_labeled.mp4'

In [ ]:
def crop_it(fl_pth, vid_pth, x):
    # deeplabcut.analyze_videos(config_path, 'CropGr.mp4', save_as_csv=True, dynamic=(True,.6,30))

    # Load tracking results generated by DeepLabCut
    tracking_data = pd.read_hdf(fl_pth)

    # Define the names of the facial features that you want to extract frames for
    feature_names = ['RightEye_Pupil','LeftEye_Pupil', 'NostrilsTop_Centre', 'OutlineTop_Mid']

    # Define the threshold for the confidence score of the facial features
    confidence_threshold = 0.9

    # Load the input video
    cap = cv2.VideoCapture(vid_pth)

    # Define the output video writer
    out_file = 'AAACrpoutput_video' + x + '.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(out_file, fourcc, 30, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    # Loop through the video frames and extract frames with facial features
    frame_number = 0
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            # Get the tracking data for the current frame
            frame_data = tracking_data.loc[frame_number]

            # Check if the desired facial features are present in the frame
            feature_present = False
            check = 0
            for feature_name in feature_names:
                if feature_name in frame_data.loc[x] and frame_data.loc[x].loc[feature_name].loc['likelihood'] > confidence_threshold:
                    check += 1
            if check==4:
                feature_present = True

            # If the desired facial features are present, save the frame to the output video
            if feature_present: #check==4
                cv2.imshow('output', frame)
                out.write(frame)

            # Display the output
            #cv2.imshow('output', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            # Increment the frame number
            frame_number += 1
        else:
            break

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [ ]:
# shutil.move('/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_video/CropGrDLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000_filtered.h5', '/home/yramakrishna/DeepLabCut/conda-environments/Codes/CropGrDLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000_filtered.h5')



In [ ]:


# final_fl_pth = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/CropGrDLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000_filtered.h5
# full_video_path

In [ ]:
# try:
#     shutil.rmtree('/home/yramakrishna/DeepLabCut/conda-environments/final_video')
# except OSError as e:
#     # If it fails, inform the user.
#     print("Error: %s - %s." % (e.filename, e.strerror))

In [ ]:
# try:
#     os.remove('/home/yramakrishna/DeepLabCut/conda-environments/Codes/Ken_Test_Longdownsampled.mp4')
# except OSError as e:
#     # If it fails, inform the user.
#     print("Error: %s - %s." % (e.filename, e.strerror))

In [ ]:
# import tkinter as tk
# from tkVideoPlayer import TkinterVideo

# root = tk.Tk()

# videoplayer = TkinterVideo(master=root, scaled=True)
# videoplayer.load(final)
# videoplayer.pack(expand=True, fill="both")

# videoplayer.play() # play the video

# root.mainloop()

In [ ]:
# cap = cv2.VideoCapture("/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4")
# ret, frame = cap.read()
# while(1):
#    ret, frame = cap.read()
#    cv2.imshow('frame',frame)
#    if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
#        cap.release()
#        cv2.destroyAllWindows()
#        break
#    cv2.imshow('frame',frame)

In [ ]:
# from moviepy.editor import *
# import pygame
 
# clip = VideoFileClip('/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4')
# clip.preview()
 
# pygame.quit()

In [ ]:
# from tkinter import *
# from tkvideo import tkvideo

# root = Tk()
# my_label = Label(root)
# my_label.pack()
# player = tkvideo("/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4", my_label, loop = 1, size = (1280,720))
# player.play()

# root.mainloop()

In [ ]:
# dlc_model = config_path
# dlc_config = deeplabcut.load_config(config_file)
# dlc_model = deeplabcut.getModel(dlc_config)


# # Load the video file
# cap = cv2.VideoCapture(video_down)

# fps = int(cap.get(cv2.CAP_PROP_FPS))

# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter('CropMFace.mp4', fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# # Loop over each frame in the video
# frame_count = 0
# video.set(cv2.CAP_PROP_POS_FRAMES, 0)

# while(cap.isOpened()):
#     ret, frame = cap.read()

#     # Break the loop when the video ends
#     if not ret:
#         break

#     # Detect the monkey face using the DeepLabCut model
#     dlc_coordinates, _ = dlc_model.predict(frame, tracking=True)
#     monkey_face_coordinates = dlc_coordinates['monkey_face']

#     # Check if monkey face is detected in the frame
#     if monkey_face_coordinates.any():
#         out.write(frame_scaled)
#         # Get the bounding box coordinates of the monkey face
#         x, y, w, h = cv2.boundingRect(monkey_face_coordinates.astype(int))

#         # Crop out the monkey face and save it to a new file
#         monkey_face = frame[y:y+h, x:x+w]
#         cv2.imwrite(f'monkey_face_{frame_count}.jpg', monkey_face)
#         frame_count += 1


#     # Break the loop if 'q' is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the video capture and close all windows
# out.release()
# cap.release()
# cv2.destroyAllWindows()


In [ ]:

# import cv2
# import numpy as np
# import deeplabcut

# # Load the DeepLabCut model
# config_file = 'path/to/config.yaml'
# dlc_model = deeplabcut.load_model(config_file)

# # Load the video file
# cap = cv2.VideoCapture('path/to/video')

# # Loop over each frame in the video
# frame_count = 0
# while(cap.isOpened()):
#     ret, frame = cap.read()
    
#     # Break the loop when the video ends
#     if not ret:
#         break
    
#     # Detect the monkey face using the DeepLabCut model
#     dlc_coordinates, _ = dlc_model.predict(frame, tracking=True)
#     monkey_face_coordinates = dlc_coordinates['monkey_face']
    
#     # Check if monkey face is detected in the frame
#     if monkey_face_coordinates.any():
#         # Get the bounding box coordinates of the monkey face
#         x, y, w, h = cv2.boundingRect(monkey_face_coordinates.astype(int))
        
#         # Crop out the monkey face and save it to a new file
#         monkey_face = frame[y:y+h, x:x+w]
#         cv2.imwrite(f'monkey_face_{frame_count}.jpg', monkey_face)
#         frame_count += 1
    
#     # Break the loop if 'q' is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the video capture and close all windows
# cap.release()
# cv2.destroyAllWindows()
